## 2.Problem Statement :  perform  Market Basket Analysis for food purchase data to uncover associations between food items.

## Data : Food purchase list

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import networkx as nx
from apyori import apriori
import warnings
warnings.filterwarnings('ignore')
df=pd.read_excel('C:/datalake/project_retail/master_dataset.xlsx', sheet_name='food_items',encoding = 'unicode_escape',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Making list from pandas Dataframe

In [4]:
transactions = []
for i in range(0, 7501):
    transactions.append([str(df.values[i,j]) for j in range(0, 20)])

In [5]:
transactions[:777]

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers',
  'meatballs',
  'eggs',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['chutney',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['turkey',
  'avocado',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan'],
 ['mineral water',
  'milk',
  'energy bar',
  'whole wheat rice',
  'green tea',
  'nan',
  'nan',
  'nan',
 

## Important terms of Markrt Basket analysis

- #### 1.Support. This says how popular an itemset is, it is number of times appear in total number of transaction. in other word we say frequency of item.
- #### 2.Confidence. This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears.
- #### 3.Lift. it is ratio of expected confidance to observed confidance.it is described as confidance of Y when item X was already known(x/y) to the confidance of Y when X item is unknown. in other words confidance of Y w.r.t. x and confiadnce of Y without X (means both are independent to each other).

## Appying Apriori algorithm and setting value for Support,Confidance,Lift

- ### Training Apriori on the dataset
- ### The choosing Hyperparameters Support,Comfidence,Lift.

In [6]:
from apyori import apriori
rules = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [7]:
results = list(rules)

In [8]:
lift = []
association = []
for i in range (0, len(results)):
    lift.append(results[:len(results)][i][2][0][3])
    association.append(list(results[:len(results)][i][0]))

In [9]:
df2 = pd.DataFrame([association, lift]).T
df2.columns = ['Association', 'Lift']
df2.sample(3)

,Association,Lift
157,"[nan, spaghetti, mineral water, tomatoes, grou...",4.37609
25,"[nan, escalope, mushroom cream sauce]",3.79083
108,"[nan, tomatoes, spaghetti, frozen vegetables]",3.49805


### Calculating Lift value for Rows

In [12]:
df2.sort_values('Lift', ascending=False).head()

,Association,Lift
150,"[nan, milk, frozen vegetables, mineral water, ...",7.98718
97,"[mineral water, milk, soup, frozen vegetables]",7.98718
149,"[nan, olive oil, milk, frozen vegetables, mine...",6.12827
96,"[mineral water, milk, olive oil, frozen vegeta...",6.12827
132,"[mineral water, whole wheat pasta, olive oil, ...",6.12827


In [13]:
df2.get_value(150, 'Association') 

['nan', 'milk', 'frozen vegetables', 'mineral water', 'soup']

 #### Milk,Soup,Mineral water,frozen vegetables are the most commom combined items who have maximum value of Lift.